##### This file creates the final csv which can be upload to the moodle named as moodle_gradeA2.csv

In [ ]:
import os 
import pandas as pd
import numpy as np 
from tabulate import tabulate
from pprint import pprint

In [ ]:
home_dir = os.getcwd()
home_dir

Scaling factors

In [ ]:
final_scaling = {'report': 1.8 / 100, 
    'correctness': 4.5 / 100, 
    'performance': 2.7 / 100,
}
final_scaling

Marks for each testid

In [ ]:
s_id, e_id = 0, 16

range_of_k = dict()
for testid in range(s_id, e_id):
    test_dir = os.path.join(home_dir, f'test{testid}')
    if os.path.exists(test_dir):
        info_file = (os.path.join(test_dir, 'task1_info.txt'))
        startk, endk = None, None
        with open(info_file, 'r') as f:
            line: str = f.readline().split(',')
            startk: int = int((line[0]).split('=')[1])
            endk: int = int((line[1]).split('=')[1])
            f.close()
        range_of_k[testid] = (startk, endk)

print (range_of_k)

weights = {'small': 50, 'medium': 25, 'large': 25}
categories = {'small': [0, 1, 2, 3, 4, 5], 'medium': [6, 10, 12, 14], 'large': [15]}
marksFor = dict()

for category, ids in categories.items():
    uniform_weight = weights[category] / len(ids)
    for testid in ids:
        marksFor[testid] = uniform_weight

subpart_marks = dict()
for testid, (startk, endk) in range_of_k.items():
    if not(testid in marksFor):
        continue
    marks = marksFor[testid]
    total_subpart = endk - startk + 1
    subpart_marks[testid] = round(marks / total_subpart, 2)

print (subpart_marks)

Read the pairs of students who are doing assignment in pair

In [ ]:
student_pair_df = pd.read_csv(os.path.join(home_dir, 'studentPairs.csv'))
student_pair_df.set_index('student1', inplace=True)

Students who need demo

In [ ]:
scan_df = pd.read_csv(os.path.join(home_dir, 'report_scores.csv'))
need_demo = scan_df[scan_df['Demo required'] == 'Need demo']['Entry Number']
need_demo.to_csv(os.path.join(home_dir, 'need_demo.csv'))

Final grades without runtime

In [ ]:
correct_df = pd.read_csv(os.path.join(home_dir, 'correctness.csv'))
correct_df = correct_df[['Entry_no', 'Total Score']]
scan_df = scan_df[['Entry Number', 'Total Score']]
# print(tabulate(scan_df, headers='keys', tablefmt='psql'))

Get kerberos from the entry number and also the group ids

In [ ]:
def get_kerberos(Entry_no: str) -> str:
    year, dept, endwith = Entry_no[2: 4], Entry_no[4: 7], Entry_no[7: ]
    kerberos = dept + year + endwith
    return kerberos.lower()

def groupid(Entry_no: str) -> str:
    try:
        return (student_pair_df.loc[Entry_no, 'kerbsids'].lower())
    except Exception as e:
        print (f'Not present in the pair form: {Entry_no}')

Make a format for comment to pass

In [ ]:
penalty_df = pd.read_csv(os.path.join(home_dir, 'demo_penalty.csv'))
penalty_df.set_index('email_address', inplace=True)

demo_students = list(penalty_df.index)
demo_students = [eno.split('@')[0] for eno in demo_students]

def get_penalty(entry_no):
    return 0.75 * penalty_df.loc[f'{entry_no}@cse.iitd.ac.in', 'penalty']

In [ ]:
correct_df = pd.read_csv(os.path.join(home_dir, 'correctness.csv'))
scan_df = pd.read_csv(os.path.join(home_dir, 'report_scores.csv'))
performance_df = pd.read_csv(os.path.join(home_dir, 'runtime_score.csv'))
# some renaming
scan_df = scan_df.replace(np.nan, 'No')
correct_df.rename(columns={'Total Score': 'TOTAL CORRECTNESS SCORE(out of 100)'}, inplace=True)
scan_df.rename(columns={'Total Score': 'TOTAL REPORT SCORE(out of 100)', 'Demo required': 'Demo required ?'}, inplace=True)
performance_df.rename(columns={'Total Score': 'TOTAL PERFORMANCE SCORE(out of 100)'}, inplace=True)
scan_df.drop(columns='Demo required ?', inplace=True)

comments = dict()


for _, row in correct_df.iterrows():
    correctness_marks = dict()
    Entry_no = row['Entry_no']
    for cols in correct_df.columns:
        correctness_marks[cols] = row[cols]
        
    comments[Entry_no] = {
        'correctness': correctness_marks,
    }
    kerberos = get_kerberos(Entry_no)
    group_id = groupid(Entry_no)
    if group_id is None:
        comments[Entry_no]['group_id'] = kerberos + '_' + kerberos
    else:
        comments[Entry_no]['group_id'] = group_id
    
# Scale marks
for _, row in scan_df.iterrows():
    report_marks = dict() 
    Entry_no = row['Entry Number']
    for cols in scan_df.columns:
        if cols == 'Entry number':
            continue
        report_marks[cols] = row[cols]
    comments[Entry_no]['report'] = report_marks

for _, row in performance_df.iterrows():
    performance_marks = dict() 
    Entry_no = row['Entry_no']
    for cols in performance_df.columns:
        if cols == 'Entry_no':
            continue
        performance_marks[cols] = row[cols] 
    comments[Entry_no]['performance'] = performance_marks

pprint(comments)

In [ ]:
moodle_csv = {'userid': [], 'marks': [], 'comments': []}
# moodle_csv['Entry_no'] = list(comments.keys())
# moodle_csv['groupid'] = [comments[e]['group_id'] for e in comments.keys()]

moodle_export_df = pd.read_excel(os.path.join(home_dir, '2202-COL380 Grades.xlsx'))
moodle_export_df.to_csv(os.path.join(home_dir, 'col380exports.csv'), header=True, index=None)
moodle_export_df = pd.read_csv(os.path.join(home_dir, 'col380exports.csv'))

for e, comment in comments.items():
    groupid = comments[e]['group_id']
    Entry_no = e
    studentA = groupid.split('_')[0]
    studentB = groupid.split('_')[1]
    penalty = 0.0
    if (studentA in demo_students):
        penalty = get_penalty(studentA)
    if (studentB in demo_students):
        penalty = get_penalty(studentB)

    comment['penalty_for_demo'] = penalty
    

    scaled_correctness_score = comment['correctness']['TOTAL CORRECTNESS SCORE(out of 100)'] * final_scaling['correctness']
    scaled_performance_score = comment['performance']['TOTAL PERFORMANCE SCORE(out of 100)'] * final_scaling['performance']
    if 'report' not in comment:
        print (f"Yes : {studentA}, {studentB}")
        comment['report'] = {}
        comment['report']['TOTAL REPORT SCORE(out of 100)'] = 0.0
    scaled_report_score = comment['report']['TOTAL REPORT SCORE(out of 100)'] * final_scaling['report']

    print (f"Before = {comment['correctness']['TOTAL CORRECTNESS SCORE(out of 100)']}, now = {scaled_correctness_score}")
    # print (scaled_correctness_score, scaled_performance_score)
    marks = scaled_report_score + max(0, scaled_correctness_score + scaled_performance_score - penalty)
    marks = round(marks, 2)
    # marks = round(comments[e]['SCALED FINAL MARKS(WITHOUT PERFORMANCE MARKS)'], 2)
    if studentA == studentB:
        moodle_csv['userid'].append(studentA)
        moodle_csv['marks'].append(marks)
        moodle_csv['comments'].append(comment)
    else:
        print (studentA, studentB)
        moodle_csv['userid'].extend([studentA, studentB])
        moodle_csv['marks'].extend([marks, marks])
        moodle_csv['comments'].extend([comment, comment])

moodle_csv    

alluserid = set(list(moodle_export_df['ID number']))
submitted = set(list(moodle_csv['userid']))

print (f'Total user on moodle avail {alluserid}, how many submitted = {submitted}')

notsubmitted = alluserid - submitted
dropcourse = submitted - alluserid 
commons = (dropcourse) and (notsubmitted)

pprint (f'len: {len(commons)}, Commons: {commons}')
pprint (f'len: {len(notsubmitted)} Not sumbitted: {notsubmitted}')
pprint (f'len: {len(dropcourse)} Dropped : {dropcourse}')

for e in notsubmitted:
    moodle_csv['userid'].append(e)
    moodle_csv['marks'].append(0)
    moodle_csv['comments'].append('Not Submitted')

moodle_csv = pd.DataFrame(moodle_csv)
moodle_csv.set_index('userid', inplace=True)

for e in dropcourse:
    moodle_csv.drop(e, inplace=True)

moodle_csv.to_csv(os.path.join(home_dir, 'moodle_gradesA2.csv'))


In [ ]:
stats_df = pd.read_csv(os.path.join(home_dir, 'moodle_gradesA2.csv'))
marks = stats_df[stats_df['comments'] != 'Not Submitted']['marks']
userids = stats_df[stats_df['comments'] != 'Not Submitted']['userid']
distribution = marks.describe()
print(distribution)
pprint (marks)

In [ ]:
topScores = pd.Series(marks.nlargest(18).unique())
print (len(topScores))
print (f'Stats for top 20: {topScores.describe()}')

topScores = pd.Series(marks.nlargest(37).unique())
print (len(topScores))
print (f'Stats for top 20: {topScores.describe()}')


In [ ]:
from scipy.stats import norm
import seaborn as sns

sns.kdeplot(list(marks))
sns.displot(list(marks), kde=True)

Stats for the correctness score

In [ ]:
correct_df = pd.read_csv(os.path.join(home_dir, 'correctness.csv'))
correct_df['Entry_no'] = correct_df['Entry_no'].map(get_kerberos)
userids = list(userids)
print (len(userids))
marks = []
for _, row in correct_df.iterrows():
    if (row['Entry_no'] in userids):
        marks.append(row['Total Score'] * final_scaling['correctness'])

marks = pd.Series(marks)
marks.describe()